In [2]:
import numpy as np
import scipy 
import pandas as pd

# 1 Problema de Fluxo de Carga
## 1.1 Conseguir matriz admitância

In [3]:
def get_admitance(R,X):
    i =1j
    return R/(R*R+X*X)-X*i/(R*R+X*X)

In [4]:
lines = pd.DataFrame([[0.0050, 0.200],
                      [0.0090, 0.100],
                      [0.0030, 0.060],
                      [0.00225, 0.100],
                      [0.0010, 0.05]],
                    columns=['r', 'x'],
                    index=['y12','y13','y13*','y23','y34'])

In [5]:
y = get_admitance(lines.r,lines.x)
y

y12      0.124922-4.996877j
y13      0.892769-9.919651j
y13*    0.831255-16.625104j
y23      0.224886-9.994940j
y34     0.399840-19.992003j
dtype: complex128

In [6]:
y['y12']+y['y13']+y['y13*']+0.54j/2+0.82j/2+0.22j/2

(1.8489456936887732-30.751631687095323j)

In [7]:
-(y['y13']+y['y13*'])

(-1.7240237698911467+26.54475473519026j)

In [8]:
y['y12']+y['y23']+0.54j/2+0.88j/2

(0.34980807518348733-14.281817013498875j)

In [9]:
y['y13']+y['y13*']+y['y34']+y['y23']+0.82j/2+0.22j/2+0.88j/2+0.44/2

(2.5687499852514177-55.57169799550459j)

In [10]:
y['y34']+0.44j/2

(0.39984006397441013-19.772003198720512j)

## 1.2 Calcular Potências

In [11]:
def get_real_power(Vi,Vm,Gim,Bim,TETAim):
    Gim = np.array(Gim)
    Bim = np.array(Bim)
    row = Vm*(Gim*np.cos(TETAim)+Bim*np.sin(TETAim))
    
    return Vi*sum(row)

In [12]:
def get_complex_power(Vi,Vm,Gim,TETAim,Bim):
    Gim = np.array(Gim)
    Bim = np.array(Bim)
    row = Vm*(Gim*np.sin(TETAim)-Bim*np.cos(TETAim))
    return Vi*sum(row)

In [13]:
V = [1, 1, 1.02, 1]

G = [[1.84,-0.125,-0.724,0],
     [0.125,0.35,-0.225,0],
     [-1.724,0.225,2.57,-0.4],
     [0,0,-0.4,0.4]]

B = [[-30.8,5,26.6,0],
     [5, -14.3, 10, 0],
     [26.6, 10, -55.6,20],
     [0, 0, 20, 19.8]]

TETAim=[0,0,0,0]

In [14]:
PQs_calculados = np.array([])

In [15]:
P2 = get_real_power(1.03,[1,1,1.02,1],G[1], 
              B[1],0)
PQs_calculados = np.append(PQs_calculados, P2)
PQs_calculados

array([0.252865])

In [16]:
P3 = get_real_power(1.03,[1,1,1.02,1],G[2], 
              B[2],0)
PQs_calculados = np.append(PQs_calculados, P3)
PQs_calculados

array([0.252865, 0.744072])

In [17]:
P4 = get_real_power(1.03,[1,1,1.02,1],G[3], 
              B[3],0)
PQs_calculados = np.append(PQs_calculados, P4)
PQs_calculados

array([ 0.252865,  0.744072, -0.00824 ])

In [18]:
Q2 = get_complex_power(1.03,[1,1,1.02,1],G[1], 
              B[1],0)
PQs_calculados = np.append(PQs_calculados, Q2)
PQs_calculados

array([ 0.252865  ,  0.744072  , -0.00824   , -0.35059436])

In [19]:
Q4 = get_complex_power(1.03,[1,1,1.02,1],G[3], 
              B[3],0)
PQs_calculados = np.append(PQs_calculados, Q4)
PQs_calculados

array([ 0.252865  ,  0.744072  , -0.00824   , -0.35059436, -0.04842253])

In [20]:
PQs_esperados = np.array([-1,-0.5,3.6,-5,-1])

In [21]:
delta = PQs_esperados - PQs_calculados
delta

array([-1.252865  , -1.244072  ,  3.60824   , -4.64940564, -0.95157747])

In [31]:
def get_H(G, B, V, teta):
    G = np.matrix(G)
    B = np.matrix(B)
    teta = np.array(teta)
    V = np.array(V)
    result = np.matrix(np.array([[]]))
    for i in range(len(G)):
        row = np.array([])
        for j in range(len(G)):
            if(i==j):
                element = -B[i,i]*V[i]-get_complex_power(V[i],V,G[i],teta,B)
            else:
                element = V[i]*V[j]*(np.array(G[j])*np.sin(teta)-np.array(B[j])*np.cos(teta))
            np.append(row,element)
            print(row)
        np.append(result,row)
        print(result)
    return result

In [32]:
get_H(G,B,V,TETAim)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


matrix([], shape=(1, 0), dtype=float64)

In [ ]:
np.matrix(B)[1,1]

In [26]:
x= np.matrix([[1,-1,-1,1],
     [1,1,1,1],
     [1,1,1,1],
     [1,1,1,1]])
def jacobian_tensorflow(x):    
    jacobian_matrix = []
    for m in range(M):
        # We iterate over the M elements of the output vector
        grad_func = tf.gradients(model.output[:, m], model.input)
        gradients = sess.run(grad_func, feed_dict={model.input: x.reshape((1, x.size))})
        jacobian_matrix.append(gradients[0][0,:])
        
    return np.array(jacobian_matrix)